# Baltimore Analytics — Neighborhood Clustering
**Project:** `baltimore-analytics`  
**Input:** `analytics.neighborhood_features`  
**Output:** `analytics.neighborhood_clusters` + `neighborhood_clusters.csv`  
**Author:** Spencer  

---
### What This Notebook Does
Uses K-means clustering to segment Baltimore's 279 neighborhoods into cohorts based on the feature matrix built in notebook 03. Cluster labels are written back to BigQuery for use in Looker Studio.

### Approach
1. Load feature matrix from BigQuery
2. Select and scale clustering features
3. Use elbow method + silhouette scores to determine optimal K
4. Fit final K-means model
5. Profile and label each cluster
6. Write results to BigQuery + CSV

---

## 0. Install Dependencies

In [ ]:
# Uncomment and run once
# %pip install scikit-learn matplotlib seaborn google-cloud-bigquery pandas pyarrow

## 1. Configuration

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import warnings
import logging

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s — %(levelname)s — %(message)s')
log = logging.getLogger(__name__)

GCP_PROJECT       = "baltimore-analytics"
ANALYTICS_DATASET = "analytics"
GCP_REGION        = "us-east1"
CREDENTIALS_PATH  = "service_account.json"

# Minimum population to include in clustering — excludes parks/industrial areas
MIN_POPULATION = 100

# K range to evaluate
K_MIN = 2
K_MAX = 10

# Random seed for reproducibility
RANDOM_STATE = 42

print("✓ Config loaded.")

## 2. Initialize Client & Load Data

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    CREDENTIALS_PATH,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
bq = bigquery.Client(project=GCP_PROJECT, credentials=credentials)

# Load feature matrix
df = bq.query(f"""
    SELECT * FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features`
    ORDER BY neighborhood
""").to_dataframe()

print(f"✓ Loaded {len(df)} neighborhoods × {len(df.columns)} features")

# Neighborhoods to exclude from residential clustering:
# - Industrial/entertainment areas with artificially inflated crime per capita
#   due to near-zero residential population
# - Institutional campuses (universities, hospitals) with atypical property patterns
# These are analyzed separately in Cell 9.
EXCLUDE_NEIGHBORHOODS = [
    # High-crime-per-capita industrial/entertainment (low residential pop)
    "Canton Industrial Area", "Fairfield Area", "Inner Harbor",
    "Pulaski Industrial Area", "Stadium/Entertainment Area",
    # Institutional campuses
    "Dunbar-Broadway", "Hopkins Bayview", "Johns Hopkins Homewood",
    "Morgan State University", "Penn-Fallsway", "Perkins",
    "University of Maryland", "Harbor Point",
]

df_full = df.copy()
df = df[
    (df["population"] >= MIN_POPULATION) &
    (~df["neighborhood"].isin(EXCLUDE_NEIGHBORHOODS))
].reset_index(drop=True)

print(f"✓ {len(df)} neighborhoods after filtering")
print(f"  Low population excluded: {df_full[df_full['population'] < MIN_POPULATION]['neighborhood'].tolist()}")
print(f"  Institutional/industrial excluded: {EXCLUDE_NEIGHBORHOODS}")


## 3. Select & Prepare Clustering Features

In [ ]:
from scipy.stats import mstats

# Per-capita and rate features preferred over raw counts to avoid population bias
CLUSTERING_FEATURES = [
    # Demographics
    "population",
    "med_age",
    "housing_vacancy_rate",
    "owner_occupancy_rate",
    # Safety
    "crime_per_capita",
    "arrest_per_capita",
    "crime_yoy_change",
    # Housing / Vacancy
    "vacant_notices_per_unit",
    "rehab_to_vacant_ratio",
    # Investment
    "permit_value_per_capita",
    "permit_recent_5yr",
    # Quality of life
    "requests_311_per_capita",
    "requests_311_blight",
    # Property values
    "median_assessed_value",
    "avg_assessed_value",
]

available = [f for f in CLUSTERING_FEATURES if f in df.columns]
missing   = [f for f in CLUSTERING_FEATURES if f not in df.columns]
if missing:
    print(f"⚠ Missing features (skipped): {missing}")

X_raw = df[available].copy()
print(f"✓ {len(available)} features selected for clustering")

# Impute missing values with column median
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X_raw)
null_counts = X_raw.isnull().sum()
if null_counts.sum() > 0:
    print(f"  Imputed nulls:")
    print(null_counts[null_counts > 0])

# Winsorize at 1st/99th percentile to prevent extreme outliers
# from dominating their own clusters
X_winsorized = mstats.winsorize(X_imputed, limits=[0.01, 0.01], axis=0)

# Standardize — K-means is distance-based, scale matters
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_winsorized)

print(f"✓ Features winsorized and scaled. Shape: {X_scaled.shape}")


## 4. Elbow Method + Silhouette Scores
Evaluate K from 2 to 10. Look for the elbow in inertia and peak in silhouette score.

In [ ]:
k_range     = range(K_MIN, K_MAX + 1)
inertias    = []
silhouettes = []

print(f"Evaluating K = {K_MIN} to {K_MAX}...\n")
print(f"{'K':>4}  {'Inertia':>12}  {'Silhouette':>12}")
print("-" * 32)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    sil = silhouette_score(X_scaled, labels)
    silhouettes.append(sil)
    print(f"{k:>4}  {km.inertia_:>12.1f}  {sil:>12.4f}")

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(list(k_range), inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('K (Number of Clusters)', fontsize=12)
ax1.set_ylabel('Inertia (Within-cluster Sum of Squares)', fontsize=12)
ax1.set_title('Elbow Method', fontsize=14)
ax1.grid(True, alpha=0.3)

ax2.plot(list(k_range), silhouettes, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('K (Number of Clusters)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score by K', fontsize=14)
ax2.grid(True, alpha=0.3)

best_k = list(k_range)[silhouettes.index(max(silhouettes))]
ax2.axvline(x=best_k, color='green', linestyle='--', label=f'Best K = {best_k}')
ax2.legend()

plt.tight_layout()
plt.savefig('elbow_silhouette.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\n✓ Best K by silhouette: {best_k}")
print(f"  Review the elbow plot and override K_FINAL below if needed.")

## 5. Top-Level Split — K=2
Fit the primary K=2 split that divides neighborhoods into Stable and Distressed groups.

In [ ]:
K_TOP = 2
km_top = KMeans(n_clusters=K_TOP, random_state=RANDOM_STATE, n_init=20)
df["top_cluster"] = km_top.fit_predict(X_scaled)

print("Top-level cluster sizes:")
print(df["top_cluster"].value_counts().sort_index())

print("\nCluster profiles:")
print(df.groupby("top_cluster")[["crime_per_capita", "vacant_notices_per_unit",
                                   "median_assessed_value", "housing_vacancy_rate",
                                   "permit_value_per_capita"]].mean().round(2))


## 6. Sub-Clustering
Sub-cluster each top-level group independently. Silhouette determines optimal K (range 2-5).

In [ ]:
def find_best_k(X, k_range=range(2, 6), random_state=42):
    """Find optimal K using silhouette score."""
    best_k, best_score = 2, -1
    scores = {}
    for k in k_range:
        if len(X) <= k:
            break
        km = KMeans(n_clusters=k, random_state=random_state, n_init=10)
        labels = km.fit_predict(X)
        score = silhouette_score(X, labels)
        scores[k] = round(score, 4)
        if score > best_score:
            best_k, best_score = k, score
    return best_k, best_score, scores


sub_cluster_col = "sub_cluster"
df[sub_cluster_col] = -1
sub_results = {}

avg_crime = df["crime_per_capita"].mean()

for top in sorted(df["top_cluster"].unique()):
    mask = df["top_cluster"] == top
    X_sub = X_scaled[mask.values]
    grp_crime = df.loc[mask, "crime_per_capita"].mean()
    label = "Stable" if grp_crime < avg_crime else "Distressed"

    print(f"\nTop cluster {top} ({label}) - {mask.sum()} neighborhoods")
    best_k, best_score, scores = find_best_k(X_sub)
    print(f"  Silhouette scores: {scores}")
    print(f"  Best K: {best_k} (score: {best_score:.4f})")

    km_sub = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=20)
    sub_labels = km_sub.fit_predict(X_sub)

    # Offset sub-cluster IDs so they dont overlap across top clusters
    offset = top * 10
    df.loc[mask, sub_cluster_col] = sub_labels + offset
    sub_results[top] = {"label": label, "best_k": best_k, "best_score": best_score}

print("\nFinal sub-cluster sizes:")
print(df[sub_cluster_col].value_counts().sort_index())


## 7. PCA Visualization

In [ ]:
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df["top_cluster"]  = df["top_cluster"].values
pca_df["sub_cluster"]  = df[sub_cluster_col].values
pca_df["neighborhood"] = df["neighborhood"].values

unique_subs = sorted(pca_df["sub_cluster"].unique())
colors = cm.nipy_spectral(np.linspace(0, 1, len(unique_subs)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

for i in sorted(pca_df["top_cluster"].unique()):
    mask = pca_df["top_cluster"] == i
    label = sub_results[i]["label"]
    ax1.scatter(pca_df.loc[mask, "PC1"], pca_df.loc[mask, "PC2"],
               label=f"Cluster {i} ({label})", s=60, alpha=0.8)
ax1.set_title("Top-Level K=2 Split", fontsize=13)
ax1.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
ax1.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
ax1.legend()
ax1.grid(True, alpha=0.3)

for i, sub in enumerate(unique_subs):
    mask = pca_df["sub_cluster"] == sub
    ax2.scatter(pca_df.loc[mask, "PC1"], pca_df.loc[mask, "PC2"],
               c=[colors[i]], label=f"Sub-cluster {sub}", s=60, alpha=0.8)
ax2.set_title("Hierarchical Sub-Clusters", fontsize=13)
ax2.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
ax2.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3)

plt.suptitle("Baltimore Neighborhood Clustering - Hierarchical K-Means", fontsize=14)
plt.tight_layout()
plt.savefig("pca_clusters.png", dpi=150, bbox_inches="tight")
plt.show()


## 8. Cluster Profiling

In [ ]:
profile = df.groupby(sub_cluster_col)[available].mean().round(3).T
print("Sub-cluster feature profiles (mean values):")
display(profile)

profile_norm = profile.apply(lambda x: (x - x.mean()) / x.std() if x.std() > 0 else x * 0, axis=1).fillna(0)
n_subs = len(df[sub_cluster_col].unique())

fig, ax = plt.subplots(figsize=(max(8, n_subs * 2), len(available) * 0.4 + 2))
sns.heatmap(profile_norm, annot=False, cmap="RdYlGn_r", center=0,
            linewidths=0.5, ax=ax, cbar_kws={"label": "Z-score (red=high, green=low)"})
ax.set_title("Sub-Cluster Feature Profiles", fontsize=14)
plt.tight_layout()
plt.savefig("cluster_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()

for sub in sorted(df[sub_cluster_col].unique()):
    hoods = df[df[sub_cluster_col] == sub]["neighborhood"].sort_values().tolist()
    top = sub // 10
    print(f"\nSub-cluster {sub} ({sub_results[top]['label']}) - {len(hoods)} neighborhoods:")
    print(", ".join(hoods))


## 9. Excluded Neighborhood Analysis
Separately cluster low-population (parks) vs institutional/industrial neighborhoods.

In [ ]:
EXCLUDE_NEIGHBORHOODS = [
    "Canton Industrial Area", "Fairfield Area", "Inner Harbor",
    "Pulaski Industrial Area", "Stadium/Entertainment Area",
    "Dunbar-Broadway", "Hopkins Bayview", "Johns Hopkins Homewood",
    "Morgan State University", "Penn-Fallsway", "Perkins",
    "University of Maryland",
]

df_excluded = df_full[
    (df_full["population"] < MIN_POPULATION) |
    (df_full["neighborhood"].isin(EXCLUDE_NEIGHBORHOODS))
].copy().reset_index(drop=True)

df_excluded["excluded_type"] = df_excluded["neighborhood"].apply(
    lambda n: "Institutional/Industrial" if n in EXCLUDE_NEIGHBORHOODS else "Low Population (Park/Open Space)"
)

print(f"Total excluded neighborhoods: {len(df_excluded)}")
print(df_excluded.groupby("excluded_type")["neighborhood"].count())

df_excluded["excluded_cluster"] = "Unclustered"

for exc_type in df_excluded["excluded_type"].unique():
    mask = df_excluded["excluded_type"] == exc_type
    subset = df_excluded[mask].copy()
    avail = [f for f in available if f in subset.columns]

    X_exc = SimpleImputer(strategy="median").fit_transform(subset[avail])
    X_exc = StandardScaler().fit_transform(X_exc)

    if len(subset) < 4:
        df_excluded.loc[mask, "excluded_cluster"] = exc_type + " - Too few to cluster"
        print(f"\n{exc_type}: only {len(subset)} neighborhoods, skipping")
        continue

    best_k, best_score, scores = find_best_k(X_exc, k_range=range(2, min(5, len(subset))))
    print(f"\n{exc_type} ({len(subset)} neighborhoods) - Best K: {best_k} (silhouette: {best_score:.4f})")

    km_exc = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
    labels = km_exc.fit_predict(X_exc)
    df_excluded.loc[mask, "excluded_cluster"] = [f"{exc_type} - Group {l}" for l in labels]

print("\nExcluded cluster assignments:")
display(df_excluded[["neighborhood", "excluded_type", "excluded_cluster",
                      "crime_per_capita", "median_assessed_value"]].sort_values("excluded_cluster"))


## 10. Label Clusters
Review the profiles in Cell 8 and fill in `SUB_CLUSTER_LABELS`. Sub-clusters in the Stable group use IDs 0x; Distressed group uses 1x.

In [ ]:
# ── EDIT THESE after reviewing Cell 8 profiles ───────────────────────────
# Sub-cluster IDs: stable group uses 0x, distressed group uses 1x

SUB_CLUSTER_LABELS = {
    0:  "Stable — Working Class Rowhouse",
    1:  "Stable — Mixed/Transitional",
    2:  "Stable — Gentrifying/Hip",
    3:  "Stable — Quiet Residential",
    4:  "Stable — Affluent North Baltimore",
    10: "Distressed — Core Disinvestment",
    11: "Distressed — Downtown/Commercial Fringe",
}
unique_subs = sorted(df[sub_cluster_col].unique())
SUB_CLUSTER_LABELS = {s: f"Cluster {s}" for s in unique_subs}

df["cluster_label"] = df[sub_cluster_col].map(SUB_CLUSTER_LABELS)
df["top_cluster_label"] = df["top_cluster"].map(
    {t: sub_results[t]["label"] for t in sub_results}
)

print("Final cluster assignments:")
print(df.groupby(["top_cluster_label", "cluster_label"])["neighborhood"]
        .count().reset_index().rename(columns={"neighborhood": "count"}).to_string(index=False))


## 11. Write Results to BigQuery

In [ ]:
# Residential neighborhoods
residential_out = df[["neighborhood", "top_cluster", "top_cluster_label",
                       sub_cluster_col, "cluster_label"]].copy()
residential_out["excluded_type"]    = "Residential"
residential_out["excluded_cluster"] = None

# Excluded neighborhoods
excluded_out = df_excluded[["neighborhood", "excluded_type", "excluded_cluster"]].copy()
excluded_out["top_cluster"]       = -1
excluded_out["top_cluster_label"] = "Excluded"
excluded_out[sub_cluster_col]     = -1
excluded_out["cluster_label"]     = excluded_out["excluded_cluster"]

# Combine and merge back to full feature matrix
combined = pd.concat([residential_out, excluded_out], ignore_index=True)
output_df = df_full.merge(combined, on="neighborhood", how="left")
output_df["_clustered_at"] = pd.Timestamp.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

dest = f"{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_clusters"
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=True
)
bq.load_table_from_dataframe(output_df, dest, job_config=job_config).result()
t = bq.get_table(dest)
print(f"✓ {t.num_rows:,} rows → {dest}")

output_df.to_csv("neighborhood_clusters.csv", index=False)
df_excluded.to_csv("excluded_neighborhood_analysis.csv", index=False)
print("✓ Saved neighborhood_clusters.csv")
print("✓ Saved excluded_neighborhood_analysis.csv")


## 12. Final Summary

In [ ]:
print("=" * 70)
print("CLUSTERING COMPLETE")
print("=" * 70)
print(f"Residential neighborhoods clustered: {len(df)}")
print(f"Excluded neighborhoods analyzed:     {len(df_excluded)}")
print(f"Total in output table:               {len(output_df)}")
print()

summary = df.groupby(["top_cluster_label", "cluster_label"]).agg(
    neighborhoods        = ("neighborhood", "count"),
    avg_crime_per_capita = ("crime_per_capita", "mean"),
    avg_vacancy_rate     = ("housing_vacancy_rate", "mean"),
    avg_assessed_value   = ("median_assessed_value", "mean"),
    avg_permit_value     = ("permit_value_per_capita", "mean"),
).round(3).reset_index()
display(summary)

print(f"""
Output files:
  BigQuery: {GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_clusters
  CSV:      neighborhood_clusters.csv
  CSV:      excluded_neighborhood_analysis.csv
  Charts:   pca_clusters.png, cluster_heatmap.png
""")


---
## Next Steps

1. **`05_looker_studio.md`** — Connect `analytics.neighborhood_clusters` to Looker Studio

**Key fields for Looker Studio:**
- `neighborhood` — join key
- `top_cluster_label` — Stable / Distressed (primary filter)
- `cluster_label` — sub-cluster cohort label
- Join to `neighborhood_boundaries` for polygon map layer

**To re-label clusters:**
Edit `SUB_CLUSTER_LABELS` in Cell 10 and re-run Cells 10-12.

**To change sub-cluster K:**
The `find_best_k()` function auto-selects — override by setting a fixed value in Cell 6.